#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

# import OS module
import os
import re

import datetime
import pandas

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)

        
# Добавление строки в лог
def add_log(sname):
    max_logs = 1000
    count_log = count_lines('log.csv')
    if count_log>=max_logs:
        f = open('log.csv','r')
        archive_file = f"log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        r = open('log.csv', 'r')
        log_data = r.read()
        r.close
        d = open(archive_file, 'w')
        d.write(log_data)
        d.close
        f.close()
        f = open('log.csv','w')
        f.close()
    f = open('log.csv','a')
    f.write(str(sname)+','+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n')
    f.close()
    return True

server.register_function(add_log, 'add_log')

#считать кол-во строк
def count_lines(filename, chunk_size=1<<13):
    with open(filename) as file:
        return sum(chunk.count('\n')
                   for chunk in iter(lambda: file.read(chunk_size), ''))
    
server.register_function(count_lines, 'count_lines')

#получить логи в каком-то промежутке времени и типа
def get_event_log(types, start_time, end_time):
    path = r"C:\Users\Андрей\OneDrive\Рабочий стол\jupyter"
    dir_list = os.listdir(path)
    result = ""
    for x in os.listdir():
        match = re.search(r"log.*\.csv", x)
        if match:
            # Prints only text file present in My Folder
            print(x)
            f = open(str(x),'r')
            # Чтение CSV-файла
            date_column = 'date'
            df = pandas.read_csv(f, delimiter=',', names=['type', date_column])

            # Фильтрация
            filtered_df = df[(df[date_column] >= start_time) & (df[date_column] <= end_time) & (df['type']  == types)]
            result = result + str(filtered_df)
            f.close()
            # Возврат отфильтрованного DataFrame
    return result

server.register_function(get_event_log, 'get_event_log')

#tet = get_event_log("type", '2022-12-19 00:00:00', '2023-12-20 18:00:00')
#print (tet)
print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...
